Задача проекта:

     Увеличить средний чек продаж. Для этого необходимо построить рекомендательную систему.
     
Цели проекта:

    1)Сформулировать предположения и гипотезы для дальнейшего построения модели.
    2)Проверить качество данных и очистить их, сгенерировать новые фичи.
    3)Определиться с параметрами модели.
    4)Построить модель, которая могла бы давать рекомендации к item/user.
    5)Решить проблему cold start (холодного запуска) для пользователей, которых нет в тренировочной выборке.
    6)Составить финальный отчёт, в котором будет указано, как предлагаемая рекомендательная система отразится на показателях бизнеса. 

Основные шаги:

    EDA.
    Формирование данных для модели.
    Построение модели.
    Тестирование модели.
    Улучшение модели.
    Production.

In [75]:
import numpy as np 
import pandas as pd 

import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import StandardScaler

# Загрузка данных.

In [76]:
# https://stackoverflow.com/questions/1592565/determine-if-variable-is-defined-in-python/1592578#1592578
try:
    train
except NameError:
    train = pd.read_csv('data/train.csv.zip', low_memory=False,thousands=',')
    train = train.drop_duplicates().reset_index(drop = True) # удалим дубликаты, если есть

try:
    test
except NameError:
    test = pd.read_csv('data/test.csv.zip', low_memory=False,thousands=',')

try:
    submission
except NameError:
    submission = pd.read_csv('data/sample_submission.csv.zip', low_memory=False)

In [77]:
# from meta_Grocery_and_Gourmet_Food.json

# try:
#     json
# except NameError:
#     json = pd.read_csv('data/json.csv.zip', low_memory=False)
# print(json.info(), json.head())

force_reload = True
infix = '_used'
suffix = '.zip'
try:
    normalized
    if force_reload:
        raise NameError('force_load')
except NameError:
    normalized = pd.read_csv(f'data/normalized{infix}.csv{suffix}', low_memory=False)
print(normalized.info(), normalized.head())
    
try:
    category
    if force_reload:
        raise NameError('force_load')
except NameError:
    category = pd.read_csv(f'data/category{infix}.csv{suffix}', low_memory=False)
print(category.info(), category.head())
    
try:
    also_view
    if force_reload:
        raise NameError('force_load')
except NameError:
    also_view = pd.read_csv(f'data/also_view{infix}.csv{suffix}', low_memory=False)
print(also_view.info(), also_view.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42003 entries, 0 to 42002
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   itemid       42003 non-null  int64 
 1   brand        41574 non-null  object
 2   description  38588 non-null  object
 3   title        42003 non-null  object
 4   main_cat     41920 non-null  object
 5   price        25786 non-null  object
 6   is_train     42003 non-null  bool  
dtypes: bool(1), int64(1), object(5)
memory usage: 2.0+ MB
None    itemid                  brand  \
0       1                 Lipton   
1       0                 Lipton   
2       2            Organo Gold   
3       3               Mae Ploy   
4       4  Harold Import Company   

                                         description  \
0  Lipton Yellow Label Tea use only the finest te...   
1  Lipton Yellow Label Teabags uses a new way to ...   
2  20 Sachets\n\nEmpty contents into cup\nPour 8o...   
3  Mae Ploy Thai g

In [4]:
# Этот участок кода требуется запустить единоразово.
# itemid_asin = train[['itemid', 'asin']]
# itemid_asin_test = test[['itemid', 'asin']]
# print(itemid_asin.info(), itemid_asin_test.info())
# itemid_asin = itemid_asin.drop_duplicates().reset_index(drop = True)
# itemid_asin_test = itemid_asin_test.drop_duplicates().reset_index(drop = True)
# print(itemid_asin.info(), itemid_asin_test.info())
# itemid_asin_concat = pd.concat([itemid_asin, itemid_asin_test]).drop_duplicates().reset_index(drop = True)
# print(itemid_asin_concat.info())
# itemid_asin_concat.to_csv('data/itemid_asin.csv')

# Шаг 1. EDA.

In [78]:
def describe(df, field_name):
    print(f"Колонка \"{field_name}\":")
    print("------")
    print("na:", df[field_name].isna().sum())
    print("уникальных значений:", len(df[field_name].unique()))
    print("------")
    print(df[field_name].value_counts())  
    
def param_data(data): # посмотрим на данные
    """dataset required parameters """
    param = pd.DataFrame({
              'dtypes': data.dtypes.values,
              'nunique': data.nunique().values,
              'isna': data.isna().sum().values,
              'loc[0]': data.loc[0].values,
              }, 
             index = data.loc[0].index)
    return param

In [79]:
pd.concat([param_data(train), param_data(test)], 
          axis=1, 
          keys = [f'↓ ОБУЧАЮЩАЯ ВЫБОРКА ↓ {train.shape}', f'↓ ТЕСТОВАЯ ВЫБОРКА ↓ {test.shape}'],  
          sort=False)

↓ ОБУЧАЮЩАЯ ВЫБОРКА ↓ (826895, 14)                      \
                                           dtypes   nunique      isna   
overall                                   float64       5.0       0.0   
verified                                     bool       2.0       0.0   
reviewTime                                 object    4580.0       0.0   
asin                                       object   41302.0       0.0   
reviewerName                               object  101207.0     164.0   
reviewText                                 object  686739.0     274.0   
summary                                    object  411451.0     166.0   
unixReviewTime                              int64    4580.0       0.0   
vote                                      float64     311.0  712944.0   
style                                      object   25892.0  398698.0   
image                                      object    6636.0  819916.0   
userid                                      int64  127448.0       0.0   
itemid                                      int64   41302.0       0.0   
rating                                    float64       2.0       0.0   
Id                                            NaN       NaN       NaN   

                                                                   \
                                                           loc[0]   
overall                                                         5   
verified                                                     True   
reviewTime                                             10 4, 2016   
asin                                                   B01CPNIEQG   
reviewerName                                              Heather   
reviewText      These are my FAVORITE spices in my collection....   
summary                           Must Add to your Spice kitchen!   
unixReviewTime                                         1475539200   
vote                                                          NaN   
style                                                         NaN   
image                                                         NaN   
userid                                                     102179   
itemid                                                      37138   
rating                                                          1   
Id                                                            NaN   

               ↓ ТЕСТОВАЯ ВЫБОРКА ↓ (285965, 11)                      \
                                          dtypes   nunique      isna   
overall                                      NaN       NaN       NaN   
verified                                    bool       2.0       0.0   
reviewTime                                object    4349.0       0.0   
asin                                      object   37876.0       0.0   
reviewerName                              object   86815.0      47.0   
reviewText                                   NaN       NaN       NaN   
summary                                      NaN       NaN       NaN   
unixReviewTime                             int64    4349.0       0.0   
vote                                     float64     198.0  246503.0   
style                                     object   18904.0  138285.0   
image                                     object    2306.0  283597.0   
userid                                     int64  109357.0       0.0   
itemid                                     int64   37876.0       0.0   
rating                                       NaN       NaN       NaN   
Id                                         int64  285965.0       0.0   

                            
                    loc[0]  
overall                NaN  
verified              True  
reviewTime      10 1, 2016  
asin            B001E5E3X0  
reviewerName     Rudys Mom  
reviewText             NaN  
summary                NaN  
unixReviewTime  1475280000  
vote                   NaN  
style                  NaN  
image                  NaN

Описание исходного датасета:

    overall - рейтинг, который поставил пользователь (значения от 1 до 5)
    verified - был ли отзыв верифицирован (значения True или False)
    reviewTime - когда был отзыв написан
    asin - предположительно, серийный номер или штрихкод
    reviewerName - имя пользователя
    reviewText - текст отзыва
    summary - сжатый отзыв
    unixReviewTime - когда был отзыв написан в формате unix
    vote - количество голосований за отзыв (значения - целые числа, представленные в виде строки)
    style - метаданные ( значения словари с описанием размера порции и аромата продукта)
    image - изображение продукта
    userid - id пользователя
    itemid - id товара
    rating - предположительно, это понравился или не понравился товар, значения (1, если overall>=4 остальное 0)
    Id - id для предсказания

In [80]:
normalized=normalized[['itemid', 'title', 'brand','description','main_cat','price']]
normalized.head()

,itemid,title,brand,description,main_cat,price
0,1,Lipton Yellow Label Tea (loose tea) - 450g,Lipton,Lipton Yellow Label Tea use only the finest te...,Grocery,$12.46
1,0,Lipton Yellow Label Finest Blend Tea Bags 100 ...,Lipton,Lipton Yellow Label Teabags uses a new way to ...,Grocery,$12.98
2,2,Organo Gold Cafe Supreme 100% Certified Ganode...,Organo Gold,20 Sachets\n\nEmpty contents into cup\nPour 8o...,Grocery,$29.90
3,3,"Mae Ploy Green Curry Paste, 14 oz",Mae Ploy,Mae Ploy Thai green curry paste.\n\nIngredient...,Grocery,NaN
4,4,"Ateco Food Coloring Kit, 6 colors",Harold Import Company,This set is a great value from one of the grea...,Amazon Home,NaN


In [81]:
normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42003 entries, 0 to 42002
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   itemid       42003 non-null  int64 
 1   title        42003 non-null  object
 2   brand        41574 non-null  object
 3   description  38588 non-null  object
 4   main_cat     41920 non-null  object
 5   price        25786 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.9+ MB


Описание дополнительного датасета normalized, извлеченного из файла meta_Grocery_and_Gourmet_Food.json:

    itemid - id товара
    title - название товара 
    brand - изготовитель
    description - описание товара
    main_cat - главная категория товара
    price - цена товара в долларах

In [82]:
#Дополнительный датасет category, извлеченный из файла meta_Grocery_and_Gourmet_Food.json:itemid - id товара,
#category - категория, к которой товар относится
print(category.head())
print(category.info())

   itemid                category  is_train
0       1  Grocery & Gourmet Food      True
1       1               Beverages      True
2       1     Coffee, Tea & Cocoa      True
3       1                     Tea      True
4       1                   Black      True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166917 entries, 0 to 166916
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   itemid    166917 non-null  int64 
 1   category  166917 non-null  object
 2   is_train  166917 non-null  bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 2.7+ MB
None


In [83]:
category

,itemid,category,is_train
0,1,Grocery & Gourmet Food,True
1,1,Beverages,True
2,1,"Coffee, Tea & Cocoa",True
3,1,Tea,True
4,1,Black,True
...,...,...,...
166912,38322,Cocoa,True
166913,38323,Grocery & Gourmet Food,True
166914,38323,Condiments & Salad Dressings,True
166915,38323,Salad Dressings,True


In [11]:
#Дополнительный датасет also_view, извлеченный из файла meta_Grocery_and_Gourmet_Food.json:itemid - id товара,
#also_view_itemid - id товара, просмотренного вместе с основным
print(also_view.head())
print(also_view.info())

   itemid  also_view_itemid  is_train
0       1              9433      True
1       1              2990      True
2       1             10750      True
3       1             19204      True
4       1              2991      True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178606 entries, 0 to 178605
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   itemid            178606 non-null  int64
 1   also_view_itemid  178606 non-null  int64
 2   is_train          178606 non-null  bool 
dtypes: bool(1), int64(2)
memory usage: 2.9 MB
None


## Анализ полей исходного датасета

In [12]:
df = train.copy()

Отберем поля 

In [13]:
selected_fields = set()

для обучающего датасета, поочередно рассматривая колонки исходного датасета:

### overall - рейтинг, который поставил пользователь

In [14]:
field_name = 'overall'
describe(df, field_name)

Колонка "overall":
------
na: 0
уникальных значений: 5
------
5.0    592278
4.0    109334
3.0     58488
1.0     36159
2.0     30636
Name: overall, dtype: int64


In [15]:
df['rating_check'] = train['overall'].apply(lambda x: 1.0 if x >= 4 else 0.0)
describe(df, 'rating_check')
describe(df, 'rating')

Колонка "rating_check":
------
na: 0
уникальных значений: 2
------
1.0    701612
0.0    125283
Name: rating_check, dtype: int64
Колонка "rating":
------
na: 0
уникальных значений: 2
------
1.0    701612
0.0    125283
Name: rating, dtype: int64


Значение этого поля отражено в колонке `rating`, не включаем его в список далее рассматриваемых

### verified

In [16]:
field_name = 'verified'
describe(df, field_name)

Колонка "verified":
------
na: 0
уникальных значений: 2
------
True     718164
False    108731
Name: verified, dtype: int64


#### TODO: Надо бы drop'нуть все неверифицированные отзывы

### asin

In [17]:
field_name = 'asin'
describe(df, field_name)

Колонка "asin":
------
na: 0
уникальных значений: 41302
------
B00BUKL666    5430
B00D3M2QP4    4733
B008QMX2SG    4611
B00R7PWK7W    2449
B000F4DKAI    2166
              ... 
B00DJLKL0Q       1
B001650VTI       1
B0010AR1C4       1
B000XK71JK       1
B00DN3T5BG       1
Name: asin, Length: 41302, dtype: int64


In [18]:
describe(df, 'itemid')

Колонка "itemid":
------
na: 0
уникальных значений: 41302
------
22208    5430
23540    4733
24556    4611
32046    2449
1919     2166
         ... 
38777       1
32616       1
5396        1
25613       1
38602       1
Name: itemid, Length: 41302, dtype: int64


Видно, что рассматриваемая колонка и `itemid`, это про одно и тоже - код товара, поэтому рассматриваемую колонку игонорируем

### reviewerName

In [19]:
field_name = 'reviewerName'
describe(df, field_name)

Колонка "reviewerName":
------
na: 164
уникальных значений: 101208
------
Amazon Customer      30612
Kindle Customer       6012
Linda                  713
John                   695
David                  597
                     ...  
Nick..                   1
Your Average Joe         1
Crystal Perritano        1
Jilana Elashvili         1
Slingo                   1
Name: reviewerName, Length: 101207, dtype: int64


In [20]:
describe(df, 'userid')

Колонка "userid":
------
na: 0
уникальных значений: 127448
------
842       418
17987     311
355       294
2024      288
2809      263
         ... 
30644       1
64828       1
104926      1
63804       1
38947       1
Name: userid, Length: 127448, dtype: int64


Поле `userid` выглядит точнее, чем рассматриваемая колонка, поэтому ее тоже игнорируем

### reviewText

In [21]:
field_name = 'reviewText'
describe(df, field_name)

Колонка "reviewText":
------
na: 274
уникальных значений: 686740
------
good                                                                                                                                                                                                                                                                                                    3019
great                                                                                                                                                                                                                                                                                                   2525
Great                                                                                                                                                                                                                                                                                                   2002
Good                     

Эту колонку оставляем для дальнейшего рассмотрения:

In [22]:
selected_fields.add(field_name)

### summary

In [23]:
field_name = 'summary'
describe(df, field_name)

Колонка "summary":
------
na: 166
уникальных значений: 411452
------
Five Stars                                       167416
Four Stars                                        26010
Three Stars                                       11682
One Star                                           5206
Two Stars                                          4824
                                                  ...  
A VERY  GOOD FOR YOU TEA!                             1
Xlear Spry gum blister tray                           1
No luck with these kefir grains - just curd!          1
NIDO Instant Full Cream Milk Powder, 900g             1
Best Jerky out there - try it for a big smile         1
Name: summary, Length: 411451, dtype: int64


Эту колонку оставляем для дальнейшего рассмотрения:

In [24]:
selected_fields.add(field_name)

### unixReviewTime

In [25]:
field_name = 'unixReviewTime'
describe(df, field_name)

Колонка "unixReviewTime":
------
na: 0
уникальных значений: 4580
------
1456790400    909
1468454400    858
1433289600    769
1453248000    764
1455926400    753
             ... 
1105142400      1
1131062400      1
1161993600      1
1142467200      1
1177113600      1
Name: unixReviewTime, Length: 4580, dtype: int64


Эту колонку оставляем для дальнейшего рассмотрения:

In [26]:
selected_fields.add(field_name)

### reviewTime

In [27]:
field_name = 'reviewTime'
describe(df, field_name)

Колонка "reviewTime":
------
na: 0
уникальных значений: 4580
------
03 1, 2016     909
07 14, 2016    858
06 3, 2015     769
01 20, 2016    764
02 20, 2016    753
              ... 
06 29, 2006      1
08 4, 2005       1
05 7, 2006       1
05 27, 2007      1
06 4, 2006       1
Name: reviewTime, Length: 4580, dtype: int64


Эта колонка про то же самое, что и `unixReviewTime`, только в другом формате. Игнорируем

### vote

In [28]:
field_name = 'vote'
describe(df, field_name)

Колонка "vote":
------
na: 712944
уникальных значений: 312
------
2.0       42820
3.0       21447
4.0       12337
5.0        7932
6.0        5480
          ...  
1060.0        1
192.0         1
208.0         1
221.0         1
680.0         1
Name: vote, Length: 311, dtype: int64


Эту колонку оставляем для дальнейшего рассмотрения:

In [29]:
selected_fields.add(field_name)

### style

In [30]:
field_name = 'style'
describe(df, field_name)

Колонка "style":
------
na: 398698
уникальных значений: 25893
------
{'Size:': ' Pack of 1'}                                                         4549
{'Package Quantity:': ' 1'}                                                     4461
{'Package Type:': ' Standard Packaging'}                                        3860
{'Size:': ' 1 Pack'}                                                            3688
{'Size:': ' 24 Count'}                                                          3615
                                                                                ... 
{'Size:': ' 1 Lb Coarse Bag'}                                                      1
{'Size:': ' 10.5 Ounce (Pack of 12)', 'Flavor:': ' Chocolate Peanut Butter'}       1
{'Size:': ' 4 Ounce', 'Flavor:': ' Lady Hannahs Whole Fruit Herbal Tea'}           1
{'Style:': ' Garden Vegetable'}                                                    1
{'Size:': ' 5 Pound', 'Flavor:': ' Whole Bean Restaurant and Cafe'}              

Эта колонка содержит очень много пропущенных значений, поэтому в дальнейшем не рассматриваем.

### image

In [31]:
field_name = 'image'
describe(df, field_name)

Колонка "image":
------
na: 819916
уникальных значений: 6637
------
['https://images-na.ssl-images-amazon.com/images/I/712zJIOTV5L._SY88.jpg']                                                                                                                                                                                                                                                                                                                                                                                      6
['https://images-na.ssl-images-amazon.com/images/I/71+Z1TA3eyL._SY88.jpg']                                                                                                                                                                                                                                                                                                                                                                                      6
['https://images-na.ssl-images-a

Эта колонка содержит очень много пропущенных значений, поэтому в дальнейшем не рассматриваем.

### userid, itemid, rating

In [32]:
print(f"max: {train.userid.max()}, unique: {len(train.userid.unique())}")

max: 127495, unique: 127448


In [33]:
print(f"max: {test.itemid.max()}, unique: {len(test.itemid.unique())}")

max: 41319, unique: 37876


Эти поля безусловно рассматриваем:

In [34]:
for field_name in ['userid', 'itemid', 'rating']:
    selected_fields.add(field_name)

# Шаг 2. Формируем данные для построения модели.

In [35]:
sdf = df[df.verified == True] # [list(selected_fields)]
sdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 718164 entries, 0 to 826894
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         718164 non-null  float64
 1   verified        718164 non-null  bool   
 2   reviewTime      718164 non-null  object 
 3   asin            718164 non-null  object 
 4   reviewerName    718024 non-null  object 
 5   reviewText      717907 non-null  object 
 6   summary         718008 non-null  object 
 7   unixReviewTime  718164 non-null  int64  
 8   vote            93604 non-null   float64
 9   style           370202 non-null  object 
 10  image           4527 non-null    object 
 11  userid          718164 non-null  int64  
 12  itemid          718164 non-null  int64  
 13  rating          718164 non-null  float64
 14  rating_check    718164 non-null  float64
dtypes: bool(1), float64(4), int64(3), object(7)
memory usage: 82.9+ MB


In [36]:
print(train.info())
print(sdf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826895 entries, 0 to 826894
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         826895 non-null  float64
 1   verified        826895 non-null  bool   
 2   reviewTime      826895 non-null  object 
 3   asin            826895 non-null  object 
 4   reviewerName    826731 non-null  object 
 5   reviewText      826621 non-null  object 
 6   summary         826729 non-null  object 
 7   unixReviewTime  826895 non-null  int64  
 8   vote            113951 non-null  float64
 9   style           428197 non-null  object 
 10  image           6979 non-null    object 
 11  userid          826895 non-null  int64  
 12  itemid          826895 non-null  int64  
 13  rating          826895 non-null  float64
dtypes: bool(1), float64(3), int64(3), object(7)
memory usage: 82.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 718164 entries, 0 to 82689

In [37]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)

In [38]:
describe(train_data, 'rating')

Колонка "rating":
------
na: 0
уникальных значений: 2
------
1.0    526498
0.0     93673
Name: rating, dtype: int64


In [39]:
# построим матрицу, где: 
    # - строкам будут соответствовать пользователи 
    # - столбцам — товары
    # - на пересечении пользователя и товара - количественная оценка степени их взаимодействия, если таковое имело место.
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                 train_data['itemid'])))

In [40]:
print(len(test_data.userid.values))
print(len(test_data.itemid.values))

206724
206724


In [41]:
ratings_coo.shape

(127496, 41320)

## item_features

In [42]:
cat=train.merge(normalized, on=['itemid'],how='left').drop_duplicates().reset_index(drop = True)

In [85]:
cat.main_cat=cat.main_cat.fillna('Other')

In [45]:
item_f = []
col = []
unique_f1 = []
for column in cat[['main_cat']]:
    col += [column]*len(cat[column].unique())
    unique_f1 += list(cat[column].unique())
for x,y in zip(col, unique_f1):
    res = str(x)+ ":" +str(y)
    item_f.append(res)
    print(res)

main_cat:Grocery
main_cat:Health & Personal Care
main_cat:Office Products
main_cat:Sports & Outdoors
main_cat:Amazon Home
main_cat:Toys & Games
main_cat:Other
main_cat:Industrial & Scientific
main_cat:All Beauty
main_cat:Tools & Home Improvement
main_cat:Baby
main_cat:Pet Supplies
main_cat:Home Audio & Theater
main_cat:Arts, Crafts & Sewing
main_cat:Camera & Photo
main_cat:Cell Phones & Accessories
main_cat:Software
main_cat:Musical Instruments


In [46]:
from lightfm.data import Dataset
# we call fit to supply userid, item id and user/item features
dataset1 = Dataset()
dataset1.fit(
        df['userid'].unique(), # all the users
        df['itemid'].unique(), # all the items
        item_features = item_f
)

In [47]:
ll = []
for column in cat[['main_cat']]:
    ll.append(column + ':')
print(ll)

['main_cat:']


In [48]:
def feature_colon_value(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['f1:1', 'f2:1', 'f3:0', 'loc:del']

    """
    result = []
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result

In [49]:
ad_subset = cat[['main_cat']]
ad_list = [x.tolist() for x in ad_subset.values]
item_feature_list = []
for item in ad_list:
    item_feature_list.append(feature_colon_value(item))
print(f'Final output: {item_feature_list[0:5]}')

Final output: [['main_cat:Grocery'], ['main_cat:Grocery'], ['main_cat:Grocery'], ['main_cat:Grocery'], ['main_cat:Grocery']]


In [50]:
item_tuple = list(zip(cat.itemid, item_feature_list))
item_tuple[0:5]

[(37138, ['main_cat:Grocery']),
 (17322, ['main_cat:Grocery']),
 (5600, ['main_cat:Grocery']),
 (30249, ['main_cat:Grocery']),
 (4349, ['main_cat:Grocery'])]

In [51]:
item_features = dataset1.build_item_features(item_tuple, normalize= False)
item_features.todense()

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [52]:
item_features.shape

(41302, 41320)

In [53]:
np.transpose(item_features).shape

(41320, 41302)

In [54]:
ratings_coo.shape

(127496, 41320)

In [55]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset1.mapping()

Добавление новых фич ухудшает показатели модели, поэтому было принято решение от них отказаться.

# Шаг 3. Строим модель и считаем roc_auc

In [56]:
NUM_THREADS = 4 #число потоков

model_prev = None
no_components_prev = None
learning_rate_prev = None

def get_model(no_components, learning_rate):
    global model_prev, no_components_prev, learning_rate_prev
    model = model_prev if model_prev is not None and no_components_prev == no_components else LightFM(loss='logistic',learning_rate=learning_rate, no_components=no_components)
    model_prev = model  
    learning_rate_prev = learning_rate
    no_components_prev = no_components
    return model

def fit_model(no_components, learning_rate, epochs):
    model = get_model(no_components, learning_rate)
    model = model.fit(ratings_coo, epochs=epochs, num_threads=NUM_THREADS)
    #model = model.fit(ratings_coo, epochs=epochs, num_threads=NUM_THREADS,item_features=np.transpose(item_features))
    return model

def get_roc_auc_score(params):   
    no_components = params["no_components"]["actual"]
    learning_rate = params["learning_rate"]["actual"]
    learning_rate_round = params["learning_rate"]["round"]
    epochs = params["epochs"]["actual"]
    print(f"model for no_components={no_components}, learning_rate={round(learning_rate, learning_rate_round)}, epochs={epochs} . . .")
    import time
    start = time.time()
    model = fit_model(no_components, learning_rate, epochs)
    print(len(test_data.userid.values), len(test_data.itemid.values))
    preds = model.predict(test_data.userid.values, test_data.itemid.values)
    roc_auc_score = sklearn.metrics.roc_auc_score(test_data.rating, preds)
    print(f"{round(time.time() - start, 1)}s, roc_auc_score: {round(roc_auc_score, 5)}")
    return roc_auc_score
    
no_components = 30
learning_rate = 0.1
epochs = 30
params = {
    "no_components": {
        "actual": 30,
    },
    "learning_rate": {
        "actual": 0.1,
        "round": 3,
    },
    "epochs": {
        "actual": 30,
    },
}
roc_auc_score = get_roc_auc_score(params)


model for no_components=30, learning_rate=0.1, epochs=30 . . .
206724 206724
101.9s, roc_auc_score: 0.70949


# Шаг 4. Подберем гиперпараметры.

In [58]:
def get_actual():
    global no_components_min, no_components, epochs_min, epochs, learning_rate_min, learning_rate
    no_components_actual = no_components_min if no_components < no_components_min else no_components
    epochs_actual = epochs_min if epochs < epochs_min else epochs
    learning_rate_actual = learning_rate_min if learning_rate < learning_rate_min else learning_rate
    return no_components_actual, learning_rate_actual, epochs_actual
    
params = {
    "epochs": {
        "val": 0,
        "min": 1,
        "step": 8,
        "step_min_abs": 1,
        "round": 0,  
    },
    "no_components": {
        "val": 0,
        "min": 1,
        "step": 128,
        "step_min_abs": 1,
        "round": 0,  
    },
    "learning_rate": {
        "val": 0.1,
        "min": 0.005,
        "step": 0.1,
        "step_min_abs": 0.005,
        "round": 3,
    },
}
        
def update_actual():
    global params
    for k, v in params.items():
        v["actual"] = v["min"] if v["val"] < v["min"] else v["val"]
        
def stat():
    global params
    s = ''
    update_actual()
    for k, v in params.items():
        if len(s) > 0:
            s = s + ", "
        s = s + f'{k}: {v["actual"]}/{v["step"]}'
    return s


delta_min = 0.00001

import time
start = time.time()
print("Start . . .")
roc_auc = 0
import math
while True: 
    for k, v in params.items():
        v["roc_auc_delta"] = 0 

    delta = 0
    delta_neg = 0
    for k, v in params.items():
        if abs(v["step"]) >= v["step_min_abs"]:
            update_actual()
            v["actual"] = v["val"] + v["step"]
            v["roc_auc_delta"] = get_roc_auc_score(params) - roc_auc
            if v["roc_auc_delta"] > delta:
                delta = v["roc_auc_delta"]
            elif v["roc_auc_delta"] < delta_neg:
                delta_neg = v["roc_auc_delta"]

    if delta >= delta_min:
        for k, v in params.items():
            if v["roc_auc_delta"] >= delta:
                v["val"] = v["val"] + v["step"]
        roc_auc = roc_auc + delta
        
        print(f"roc_auc: {round(roc_auc, 5)}, delta: {round(delta, 5)} for {stat()}")   
    else:
        need_stop_by_min_abs = True
        for k, v in params.items():
            if abs(v["step"]) > v["step_min_abs"]:
                need_stop_by_min_abs = False
                break
        if need_stop_by_min_abs:
            print("Stopped by step_min_abs")
            break 
        elif -delta_neg < delta_min:
            print(f"Stopped by delta_min: -delta_neg={-delta_neg}")
            break
        else:
            did_change_step = False
            for k, v in params.items():
                if abs(v["step"]) > v["step_min_abs"] and v["roc_auc_delta"] <= delta_neg:
                    v["step"] = -round(math.copysign(abs(v["step"])/2, v["step"]), v["round"])
                    if v["round"] == 0:
                        v["step"] = int(v["step"])              
                    print(f'{k}_step: {v["step"]}')
                    did_change_step = True
                    break
            if not did_change_step:
                print("Stopped by step_min_abs")
                break
            
print(f"{round(time.time() - start, 1)}s, Ready: roc_auc={round(roc_auc, 5)} for {stat()}")




Start . . .
model for no_components=1, learning_rate=0.1, epochs=8 . . .
206724 206724
3.3s, roc_auc_score: 0.70084
model for no_components=128, learning_rate=0.1, epochs=1 . . .
206724 206724
13.3s, roc_auc_score: 0.62385
model for no_components=1, learning_rate=0.2, epochs=1 . . .
206724 206724
0.5s, roc_auc_score: 0.64655
roc_auc: 0.70084, delta: 0.70084 for epochs: 8/8, no_components: 1/128, learning_rate: 0.1/0.1
model for no_components=1, learning_rate=0.1, epochs=16 . . .
206724 206724
6.5s, roc_auc_score: 0.70161
model for no_components=128, learning_rate=0.1, epochs=8 . . .
206724 206724
103.2s, roc_auc_score: 0.71663
model for no_components=1, learning_rate=0.2, epochs=8 . . .
206724 206724
3.4s, roc_auc_score: 0.71337
roc_auc: 0.71663, delta: 0.01579 for epochs: 8/8, no_components: 128/128, learning_rate: 0.1/0.1
model for no_components=128, learning_rate=0.1, epochs=16 . . .
206724 206724
205.1s, roc_auc_score: 0.7291
model for no_components=256, learning_rate=0.1, epochs=8

# Шаг 5. Строим модель с подобранными гиперпараметрами, считаем roc_auc

In [60]:
update_actual()
no_components = params["no_components"]["actual"]
learning_rate = params["learning_rate"]["actual"]
epochs = params["epochs"]["actual"]

# для всего train'а:
# no_components = 128
# epochs = 7
# learning_rate = 0.225

# для verified его части:
# no_components = 128
# epochs = 10
# learning_rate = 0.187

# для verified его части (другой вариант):
# no_components = 128
# epochs = 6
# learning_rate = 0.275

model = fit_model(no_components, learning_rate, epochs)
preds = model.predict(test_data.userid.values, test_data.itemid.values)
roc_auc_score = sklearn.metrics.roc_auc_score(test_data.rating,preds)
print(f'roc_auc_score: {round(roc_auc_score, 5)} for {stat()}')



roc_auc_score: 0.7349 for epochs: 4/-1, no_components: 127/-1, learning_rate: 0.4/-0.05


In [61]:
preds = model.predict(test.userid.values, test.itemid.values)
preds.min(), preds.max()

(-19.396608352661133, 18.412948608398438)

In [62]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()


In [63]:
normalized_preds.min(), normalized_preds.max()


(0.0, 1.0)

In [64]:
normalized_preds

array([0.56179147, 0.54638915, 0.52701293, ..., 0.5290977 , 0.58585374,
       0.58937096])

In [65]:
submission['rating'] = normalized_preds
submission.to_csv('submission_fm.csv', index=False)

In [68]:
submission.head()

,Id,rating
0,0,0.561791
1,1,0.546389
2,2,0.527013
3,3,0.529603
4,4,0.549810


# Шаг 6. Production.

Посмотреть production-сервис и дополнительные описания по нему можно по ссылке: 

https://github.com/yurybikuzin/skillfactory_rds4_recommendation_challenge/blob/master/README.md

## Финальный отчет:

   Получилось построить модель и создать сервис для пользователей, который рекомендует товары исходя из того, какие товары были добавлены в корзину. 
    
   В перспективе, это позволит увеличить прибыль, за счет покупок рекомендованных товаров, которые изначально пользователь не планировал покупать.